# 1. Create a file with the peaks and trough from all the GP LCs

Python 3.7

In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sbn
import datetime # Get the current date and time

#--------------------------------------------------------60
# Import my functions

%cd /Users/arturo/Dropbox/Research/Codes_Mains/Python/MyFunctions/github/MyPyFuns/SNeIa

import mytoolsSNe

# Go back where this notebook is located:
%cd /Users/arturo/Dropbox/Research/Statistics/ML/Projects/RAISIN_ML/Codes/github/MLSNeLC

#--------------------------------------------------------60
code_created_by = 'Arturo_Avelino'
# On date: 2019.06.25 (yyyy.mm.dd)
code_name = '1_Extract_peaks_trough_GPLCs.ipynb'
code_version = '0.0.11'
code_last_update = '2019.08.14'

/Users/arturo/Dropbox/Research/Codes_Mains/Python/MyFunctions/github/MyPyFuns/SNeIa
/Users/arturo/Dropbox/Research/Statistics/ML/Projects/RAISIN_ML/Codes/github/MLSNeLC


In [ ]:
# mytoolsSNe.index2day(70)

In [ ]:
# mytoolsSNe.is_number('5')

In [ ]:
# mytoolsSNe.sn_name('sn1999ee__U_45_B_1_CSP_J_GP_mean_sigma_Filled_norma.dat')

In [ ]:
##############################################################################80

### USER

In [2]:
# info just to write in the output text file
band = 'J'

#---------------------------------------------------------60

# List with the name of the SNe to extract their features
filename = 'list_SNe.txt'

# Location of 'filename' and also of the GP LC files.
dirfile = '/Users/arturo/Dropbox/Research/Statistics/ML/Projects/RAISIN_ML/data/data_PCA/'

list_sne = np.genfromtxt(dirfile+filename, dtype=['S60'])

#---------------------------------------------------------60

dir_save_output = '/Users/arturo/Dropbox/Research/Statistics/ML/\
Projects/RAISIN_ML/data/data_PCA/'

In [ ]:
list_sne[:3]

-----

## Automatic

### Useful functions

Main function to find the peak and trough in the GP LCs

In [3]:
def find_peak_trough(lcgp_np, feature, phase_range):
    """
    Function to find the peak or trough in a Gaussian-Process light curve (LC).
    By "find" I mean to determine the phase, magnitude, err_magnitude, and
    index in the array, of the peak or trough.

    - lcgp_np (numpy array with 3 columns):
    
        - column 1: phase
        - column 2: magnitude
        - column 3: err_magnitude

    - feature (string): Options = ('peak', 'trough', 'at_Bmax').
    
        The 'peak' can be either the first, second, or any other peak in the LC.
        This parameter helps to define whether use 'min' or 'max' functions to
        search for the peaks or the trough, respetively.
        If feature='peak' then it is used the 'min' function to search for the
        -brightest- magnitude. And if feature='trough' then 'max' function is used
        to search for the -dimmest- magnitude in the specified phase range of
        the LC.
        
        The 'at_Bmax' corresponds to the magnitude at t_Bmax, i.e., phase=0.
        In this case the values of the 'phase_range' are irrelevants, i.e, I
        can give any values as entries; they will not be taken into account, 
        because the magnitude at t_Bmax is found directly from the main file.

    - phase_range (list with 2 elements):

    This function works in either normalized, absolute mag, or apparent
    magnitude LCs. Also, it works for LCs of any band (e.g, B, V, I, Y, J, H, K)

    """

    # Define a numpy array of magnitudes from the specified phase range
    mag_search_np = lcgp_np[mytoolsSNe.phase2index(phase_range[0]):
                            mytoolsSNe.phase2index(phase_range[1]),1]

    #---------------------------------------------------
    #  Find the peak or trough

    if feature == 'peak':

        # Find the -brightest- magnitude in the specified phase range
        mag_feat = min(mag_search_np)

        index_feat_int = mag_search_np.argmin()

    if feature == 'trough':

        # Find the -dimmest- magnitude in the specified phase range
        mag_feat = max(mag_search_np)

        index_feat_int = mag_search_np.argmax()


    if feature == 'at_Bmax': # Find the magnitude at tBmax

        line_tBmax = 70 # line number where the zero phase data is located.
        mag_feat = lcgp_np[line_tBmax, 1]

    #---------------------------------------------------
    #  Determine the index, day and e_mag of the feature.

    if (feature == 'peak' or feature == 'trough'):
        index_feat = index_feat_int + mytoolsSNe.phase2index(phase_range[0])
        phase_feat = mytoolsSNe.index2phase(index_feat)
        e_mag_feat = lcgp_np[index_feat][2]

    elif feature == 'at_Bmax':
        index_feat = line_tBmax
        phase_feat = lcgp_np[line_tBmax, 0]
        e_mag_feat = lcgp_np[line_tBmax, 2]

    return np.array([index_feat, phase_feat, mag_feat, e_mag_feat])



In [ ]:
##############################################################################80

---

### Loop to read and extract the values of peaks and the trough

In [ ]:
# Features to extract from the GP LC files

if band == 'Y':
    feature_list = ['peak', 'trough', 'peak', 'at_Bmax']
    feature_range = [[-7,0], [8, 20], [20,40], [-1,1]]
    
elif band == 'J':
    feature_list = ['peak', 'trough', 'peak', 'at_Bmax']
    feature_range = [[-7,0], [8, 22], [20,40], [-1,1]]
    
elif band == 'H':
    feature_list = ['peak', 'trough', 'peak', 'at_Bmax']
    feature_range = [[-7,0], [6, 18], [18,35], [-1,1]]
    
elif band == 'K':
    feature_list = ['peak', 'trough', 'peak', 'at_Bmax']
    feature_range = [[-7,0], [7, 19], [18,36], [-1,1]]

Loop to read, extract and write the values of peaks and the trough

In [ ]:
# Loop to read, extract and write the values of peaks and the trough

#-----------------------------------------------------------------------------80
# Open the text file to write the peaks and trough

textfile_1 = open(dir_save_output+'1_lc_features_%s_band.dat'%band,'w')

textfile_1.write('#         LC peaks and trough (%s band)\n'%band)
textfile_1.write('#\n')
textfile_1.write('# Gaussian-Processes light-curve files located at:\n')
textfile_1.write('# %s\n'%dirfile)
    
# Define the header text
now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y.%m.%d (yyyy.mm.dd); %H:%M hrs.")
text_Date   = '# On date: %s\n'%text_timenow
text_Author = '# Data table created by: %s\n'%code_created_by
text_script = '# Script used: %s (version %s | last update: %s)\n'%(
    code_name, code_version, code_last_update)
text_line = '#'+'-'*57 + '60\n'

# Write the header text
textfile_1.write(text_line);
textfile_1.write(text_Author); textfile_1.write(text_Date); 
textfile_1.write(text_script);
textfile_1.write(text_line);

textfile_1.write('#                        1st_peak                               \
trough                                 2nd_peak                                \
At_Bmax\n')

textfile_1.write('#sn_name,                \
index,  phase,    mag,       e_mag,    \
index,  phase,    mag,       e_mag,    \
index,  phase,    mag,       e_mag,    \
index,  phase,    mag,       e_mag,')

# textfile_1.close()

#-----------------------------------------------------------------------------80

count_sn_good = 0

# Loop over the SNe list:
for i1 in range(len(list_sne)):

    # Read the file name:
    # sn_filename = str(list_sne['f0'][i1])  # For python 2.7
    sn_filename = str(list_sne['f0'][i1])[2:-1]  # For python 3.7
    print(sn_filename)
    
    # Upload the data:
    GP_data = np.genfromtxt(dirfile+sn_filename)

    # Write the supernova name:
    textfile_1.write('\n%s,   '%sn_filename[:22])

    # Loop over features to find ('peak', 'trough', 'peak', 'at_Bmax')
    for i2 in range(len(feature_list)):

        feat_int1 = find_peak_trough(GP_data, feature=feature_list[i2], 
                     phase_range=feature_range[i2])

        index_out = feat_int1[0]
        day_out = feat_int1[1]
        mag_out = feat_int1[2]
        emag_out = feat_int1[3]

        textfile_1.write('%-3i,  %5.1f,  %9.5f,  %9.5f,   '%(
            index_out, day_out, mag_out, emag_out))
        
    count_sn_good += 1

#---------------------------------------------------------60

textfile_1.close()
print('\n# ---- %s SNe written ----'%count_sn_good)

In [ ]:
textfile_1.close(); textfile_1.close(); textfile_1.close(); 
textfile_1.close(); textfile_1.close(); textfile_1.close(); 

In [ ]:
##############################################################################80

---

# 2. Create a table with the differences in magnitude and time among features

In [4]:
import numpy as np
import pandas as pd
import datetime

In [5]:
# Directory to save the output

dir_save_output = '/Users/arturo/Dropbox/Research/Statistics/ML/Projects/\
RAISIN_ML/data/data_PCA/'

In [6]:
# Define which and where is the the CSV version of the original file:

file_features = '2_lc_features_J_band.csv'

dir_file_features = '/Users/arturo/Dropbox/Research/Statistics/ML/Projects/\
RAISIN_ML/data/data_PCA/'

# Upload the CSV file.
data_pd = pd.read_csv(dir_file_features+file_features)

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
data_pd.head()

32 SNe in this file.


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,phase.2,mag.2,e_mag.2,index.3,phase.3,mag.3,e_mag.3
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,33.0,0.85892,0.02239,70,0.0,0.0,0.0
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,27.0,1.00576,0.06707,70,0.0,0.0,0.0
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,31.0,1.13574,0.11078,70,0.0,0.0,0.0
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,31.5,0.95267,0.02714,70,0.0,0.0,0.0
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,30.0,1.07393,0.04652,70,0.0,0.0,0.0


### Create new columns with the difference in **magnitudes**:

In [7]:
# Difference in magnitude between 1st peak and trough:
data_pd['pk1-trh'] = data_pd['mag'] - data_pd['mag.1']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

32 SNe in this file.


In [8]:
# Difference in magnitude between 2nd peak and trough:
data_pd['pk2-trh'] = data_pd['mag.2'] - data_pd['mag.1']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

32 SNe in this file.


In [9]:
# Difference in magnitude between 1st and 2nd peak:
data_pd['pk1-pk2'] = data_pd['mag'] - data_pd['mag.2']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

32 SNe in this file.


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,phase.2,mag.2,e_mag.2,index.3,phase.3,mag.3,e_mag.3,pk1-trh,pk2-trh,pk1-pk2
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,33.0,0.85892,0.02239,70,0.0,0.0,0.0,-1.73064,-0.68689,-1.04375
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,27.0,1.00576,0.06707,70,0.0,0.0,0.0,-1.77136,-0.63832,-1.13304
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,31.0,1.13574,0.11078,70,0.0,0.0,0.0,-2.09020,-0.87915,-1.21105
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,31.5,0.95267,0.02714,70,0.0,0.0,0.0,-1.44663,-0.40545,-1.04118
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,30.0,1.07393,0.04652,70,0.0,0.0,0.0,-1.81848,-0.60599,-1.21249


In [10]:
# Difference in magnitude between mag at tBmax and 1st peak:
data_pd['mg0-pk1'] = data_pd['mag.3'] - data_pd['mag']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

32 SNe in this file.


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,...,mag.2,e_mag.2,index.3,phase.3,mag.3,e_mag.3,pk1-trh,pk2-trh,pk1-pk2,mg0-pk1
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,...,0.85892,0.02239,70,0.0,0.0,0.0,-1.73064,-0.68689,-1.04375,0.18483
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,...,1.00576,0.06707,70,0.0,0.0,0.0,-1.77136,-0.63832,-1.13304,0.12728
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,...,1.13574,0.11078,70,0.0,0.0,0.0,-2.09020,-0.87915,-1.21105,0.07531
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,...,0.95267,0.02714,70,0.0,0.0,0.0,-1.44663,-0.40545,-1.04118,0.08851
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,...,1.07393,0.04652,70,0.0,0.0,0.0,-1.81848,-0.60599,-1.21249,0.13856


In [34]:
# Difference in magnitude between mag at tBmax and 2st peak:
data_pd['pk2-mg0'] = data_pd['mag.2'] - data_pd['mag.3']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

Shape of the dataframe: (32, 25).


In [35]:
# Difference in magnitude between mag at tBmax and trough:
data_pd['trh-mg0'] = data_pd['mag.1'] - data_pd['mag.3']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

Shape of the dataframe: (32, 26).


### Create new columns with the difference in **phase**:

In [13]:
# Difference in phase between tBmax and 1st peak:
data_pd['P0-Ppk1'] = data_pd['phase.3'] - data_pd['phase']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

32 SNe in this file.


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,...,e_mag.2,index.3,phase.3,mag.3,e_mag.3,pk1-trh,pk2-trh,pk1-pk2,mg0-pk1,P0-Ppk1
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,...,0.02239,70,0.0,0.0,0.0,-1.73064,-0.68689,-1.04375,0.18483,4.5
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,...,0.06707,70,0.0,0.0,0.0,-1.77136,-0.63832,-1.13304,0.12728,2.5
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,...,0.11078,70,0.0,0.0,0.0,-2.09020,-0.87915,-1.21105,0.07531,2.0
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,...,0.02714,70,0.0,0.0,0.0,-1.44663,-0.40545,-1.04118,0.08851,4.0
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,...,0.04652,70,0.0,0.0,0.0,-1.81848,-0.60599,-1.21249,0.13856,4.5


In [24]:
# Difference in phase between tBmax and 1st peak:
data_pd['Ppk2-P0'] = data_pd['phase.2'] - data_pd['phase.3']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

Shape of the dataframe: (32, 23).


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,...,index.3,phase.3,mag.3,e_mag.3,pk1-trh,pk2-trh,pk1-pk2,mg0-pk1,P0-Ppk1,Ppk2-P0
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,...,70,0.0,0.0,0.0,-1.73064,-0.68689,-1.04375,0.18483,4.5,33.0
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,...,70,0.0,0.0,0.0,-1.77136,-0.63832,-1.13304,0.12728,2.5,27.0
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,...,70,0.0,0.0,0.0,-2.09020,-0.87915,-1.21105,0.07531,2.0,31.0
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,...,70,0.0,0.0,0.0,-1.44663,-0.40545,-1.04118,0.08851,4.0,31.5
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,...,70,0.0,0.0,0.0,-1.81848,-0.60599,-1.21249,0.13856,4.5,30.0


In [29]:
# Difference in phase between tBmax and trough:
data_pd['Ptrh-P0'] = data_pd['phase.1'] - data_pd['phase.3']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
data_pd.head()

Shape of the dataframe: (32, 24).


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,...,phase.3,mag.3,e_mag.3,pk1-trh,pk2-trh,pk1-pk2,mg0-pk1,P0-Ppk1,Ppk2-P0,Ptrh-P0
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,...,0.0,0.0,0.0,-1.73064,-0.68689,-1.04375,0.18483,4.5,33.0,16.0
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,...,0.0,0.0,0.0,-1.77136,-0.63832,-1.13304,0.12728,2.5,27.0,14.0
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,...,0.0,0.0,0.0,-2.09020,-0.87915,-1.21105,0.07531,2.0,31.0,17.5
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,...,0.0,0.0,0.0,-1.44663,-0.40545,-1.04118,0.08851,4.0,31.5,14.5
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,...,0.0,0.0,0.0,-1.81848,-0.60599,-1.21249,0.13856,4.5,30.0,12.5


In [41]:
# Difference in phase between trough and 1st peak:
data_pd['Ptrh-Ppk1'] = data_pd['phase.1'] - data_pd['phase']

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

Shape of the dataframe: (32, 27).


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,...,pk1-trh,pk2-trh,pk1-pk2,mg0-pk1,P0-Ppk1,Ppk2-P0,Ptrh-P0,pk2-mg0,trh-mg0,Ptrh-Ppk1
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,...,-1.73064,-0.68689,-1.04375,0.18483,4.5,33.0,16.0,0.85892,1.54581,20.5
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,...,-1.77136,-0.63832,-1.13304,0.12728,2.5,27.0,14.0,1.00576,1.64408,16.5
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,...,-2.09020,-0.87915,-1.21105,0.07531,2.0,31.0,17.5,1.13574,2.01489,19.5
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,...,-1.44663,-0.40545,-1.04118,0.08851,4.0,31.5,14.5,0.95267,1.35812,18.5
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,...,-1.81848,-0.60599,-1.21249,0.13856,4.5,30.0,12.5,1.07393,1.67992,17.0


In [42]:
# Difference in phase between trough and 2nd peak:
data_pd['Ppk2-Ptrh'] = data_pd['phase.2'] - data_pd['phase.1'] 

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

Shape of the dataframe: (32, 28).


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,...,pk2-trh,pk1-pk2,mg0-pk1,P0-Ppk1,Ppk2-P0,Ptrh-P0,pk2-mg0,trh-mg0,Ptrh-Ppk1,Ppk2-Ptrh
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,...,-0.68689,-1.04375,0.18483,4.5,33.0,16.0,0.85892,1.54581,20.5,17.0
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,...,-0.63832,-1.13304,0.12728,2.5,27.0,14.0,1.00576,1.64408,16.5,13.0
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,...,-0.87915,-1.21105,0.07531,2.0,31.0,17.5,1.13574,2.01489,19.5,13.5
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,...,-0.40545,-1.04118,0.08851,4.0,31.5,14.5,0.95267,1.35812,18.5,17.0
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,...,-0.60599,-1.21249,0.13856,4.5,30.0,12.5,1.07393,1.67992,17.0,17.5


In [43]:
# Difference in phase between 1st and 2nd peak:
data_pd['Ppk2-Ppk1'] = data_pd['phase.2'] - data_pd['phase'] 

print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
# data_pd.head()

Shape of the dataframe: (32, 29).


,#sn_name,index,phase,mag,e_mag,index.1,phase.1,mag.1,e_mag.1,index.2,...,pk1-pk2,mg0-pk1,P0-Ppk1,Ppk2-P0,Ptrh-P0,pk2-mg0,trh-mg0,Ptrh-Ppk1,Ppk2-Ptrh,Ppk2-Ppk1
0,sn1999ee__U_45_B_1_CSP,61,-4.5,-0.18483,0.02460,102,16.0,1.54581,0.02686,136,...,-1.04375,0.18483,4.5,33.0,16.0,0.85892,1.54581,20.5,17.0,37.5
1,sn2001bt__B_22_V_2_Oth,65,-2.5,-0.12728,0.03414,98,14.0,1.64408,0.07251,124,...,-1.13304,0.12728,2.5,27.0,14.0,1.00576,1.64408,16.5,13.0,29.5
2,sn2001cz__B_14_V_1_Oth,66,-2.0,-0.07531,0.03888,105,17.5,2.01489,0.11679,132,...,-1.21105,0.07531,2.0,31.0,17.5,1.13574,2.01489,19.5,13.5,33.0
3,sn2001el__U_20_B_4_Oth,62,-4.0,-0.08851,0.02130,99,14.5,1.35812,0.03547,133,...,-1.04118,0.08851,4.0,31.5,14.5,0.95267,1.35812,18.5,17.0,35.5
4,sn2002dj__U_13_B_1_Oth,61,-4.5,-0.13856,0.03986,95,12.5,1.67992,0.07259,130,...,-1.21249,0.13856,4.5,30.0,12.5,1.07393,1.67992,17.0,17.5,34.5


In [ ]:
print('Shape of the dataframe: %s.'%(str(data_pd.shape)))
data_pd.head()

In [ ]:
#-----------------------------------------------------------------------------80

### Write a text file with the features

In [84]:
textfile_1= open(dir_save_output+'3_lc_features_.dat', 'w')

textfile_1.write('#     Differences in magnitude and time among peaks and trough\n')
# textfile_1.write('# Header names of the new columns:')
# textfile_1.write('       pk1_trh,    pk2_trh,    pk1_pk2,   mg0_pk1,   \
# pk2_mg0,   trh_mg0,   P0_Ppk1,    Ppk2_P0,    Ptrh_P0,  Ptrh_Ppk1,  Ppk2_Ptrh,  Ppk2_Ppk1\n')

# Counter to use as index to read each row of the dataframe.
df_counter = 0

# Count the number of rows in the '1_lc_features_J_band.dat' file
counter_line = 0

#-----------------------------------------------------------------------------80

# Loop over each line of the text file.
for line_1 in open(dir_file_features+'1_lc_features_J_band.dat'):

    # Number or rows in the '1_lc_features_J_band.dat' file to skip when
    # reading it file because they are just the header rows.
    rows_header_skip = 10

    # Number of rows at the end of the '1_lc_features_J_band.dat' file to skip
    line_tail_skip = rows_header_skip + len(data_pd)
    
    # Write the all the header lines of the original file:
    if counter_line < rows_header_skip:

        # print('%s'%line_1)
        textfile_1.write('%s'%(line_1))
        
    # Write the header names of the new columns:
    elif counter_line == rows_header_skip:
        # Write the header names of the original columns:
        textfile_1.write('%s'%(line_1[:-2]))
        
        # Write the header names of the new columns:
        textfile_1.write('       pk1_trh,    pk2_trh,    pk1_pk2,   mg0_pk1,    \
pk2_mg0,   trh_mg0,   P0_Ppk1,    Ppk2_P0,    Ptrh_P0,  Ptrh_Ppk1,  Ppk2_Ptrh,  \
Ppk2_Ppk1\n')

    # Append the new values at the end of each row while not reaching the
    # end of the file:
    elif counter_line<=line_tail_skip:

        # print (df_counter)

        #------ Magnitude ------

        # Difference in magnitude between 1st peak and trough:
        pk1_trh = data_pd.iloc[df_counter]['pk1-trh']

        # Difference in magnitude between 2nd peak and trough:
        pk2_trh = data_pd.iloc[df_counter]['pk2-trh']

        # Difference in magnitude between 1st and 2nd peak:
        pk1_pk2 = data_pd.iloc[df_counter]['pk1-pk2']

        # Difference in magnitude between mag at tBmax and 1st peak:
        mg0_pk1 = data_pd.iloc[df_counter]['mg0-pk1']

        # Difference in magnitude between mag at tBmax and 2st peak:
        pk2_mg0 = data_pd.iloc[df_counter]['pk2-mg0']

        # Difference in magnitude between mag at tBmax and trough:
        trh_mg0 = data_pd.iloc[df_counter]['trh-mg0']

        #------ Phase ------

        # Difference in phase between tBmax and 1st peak:
        P0_Ppk1 = data_pd.iloc[df_counter]['P0-Ppk1']

        # Difference in phase between tBmax and 1st peak:
        Ppk2_P0 = data_pd.iloc[df_counter]['Ppk2-P0']

        # Difference in phase between tBmax and trough:
        Ptrh_P0 = data_pd.iloc[df_counter]['Ptrh-P0']

        # Difference in phase between trough and 1st peak:
        Ptrh_Ppk1 = data_pd.iloc[df_counter]['Ptrh-Ppk1']

        # Difference in phase between trough and 2nd peak:
        Ppk2_Ptrh = data_pd.iloc[df_counter]['Ppk2-Ptrh']

        # Difference in phase between 1st and 2nd peak:
        Ppk2_Ppk1 = data_pd.iloc[df_counter]['Ppk2-Ppk1']

        #--------------------------------------------

        # Write values at the end of each line:
        textfile_1.write('%s,   \
%4.5f,   %4.5f,   %4.5f,   \
%8.5f,   %4.5f,   %4.5f,   \
%4.5f,   %4.5f,   %4.5f,   \
%4.5f,   %8.5f,   %4.5f\n'%(
            line_1[:-5], 
            pk1_trh, pk2_trh, pk1_pk2,
            mg0_pk1, pk2_mg0, trh_mg0,
            P0_Ppk1, Ppk2_P0, Ptrh_P0,
            Ptrh_Ppk1, Ppk2_Ptrh, Ppk2_Ppk1))

        df_counter+=1 # Update counter

    counter_line+=1 # Update counter

#-----------------------------------------------------------------------------80

textfile_1.close()

In [85]:
textfile_1.close();textfile_1.close();textfile_1.close();
textfile_1.close();textfile_1.close();textfile_1.close();